In [1087]:
import pandas as pd 
import os 
import numpy as np
import datetime

In [1088]:
def mascara(df, coluna,n_digitos):
    df[coluna] = df[coluna].astype(str)
    df[coluna] = str("000") + df[coluna]
    df[coluna] = df[coluna].astype(str).str[-n_digitos:] 
    return df

In [1089]:
#Path cto
path_cto = r"E:\Qualidade_Florestal\01- SÃO PAULO\03- Colheita\00 - Bases de Trabalho\Bases de Apoio\CTO.xlsx"

In [1090]:
#Path estoque

path_estoque = r'E:\Qualidade_Florestal\01- SÃO PAULO\03- Colheita\00 - Bases de Trabalho\Bases de Apoio\estoque_de_madeira.xlsx'

In [1091]:
#Path pas

path_pas = 'F:\Planejamento_Florestal\Equipe\Pedro de Oliveira Sousa Castro\Silvicultura\Automação Plano de Demandas\Bases\PAS.xlsx'

In [1092]:
#base de retalhonamento
ret = pd.read_excel('retalhonamento.xlsx', dtype={'objeto de locação origem' : 'object', 'objeto de locação retalhonado': 'object'})

ret = ret[['objeto de locação origem', 'objeto de locação retalhonado']]

#Removendo duplicatas de objeto de locação origem
ret = ret.drop_duplicates(['objeto de locação origem'], keep= 'first')

In [1093]:
#base de CTO
cto = pd.read_excel(path_cto, dtype={'id_projeto': 'object', 'cd_talhao': 'object'})

cto['objeto de locação'] = cto['id_projeto'].astype(str) + cto['cd_talhao'].astype(str)

In [1094]:
#base bruta

bd = pd.read_excel("Plano Formação_regionais_v2_Marcos.xlsx", sheet_name='BD', dtype={'Id Projeto':'object', 'Talhão': 'object', 'CHAVE': 'object'})

bd = bd[pd.notna(bd['RF PLANTIO_OFICIAL'])]

In [1095]:
#Estoque 
estoque = pd.read_excel(path_estoque, dtype={'Talhão': 'object'})

estoque['id_fazenda'] = estoque['Talhão'].str[2:6]

estoque['id_talhao'] = estoque['Talhão'].str[7:]

estoque['objeto de locação'] = estoque['id_fazenda'].astype(str) + estoque['id_talhao'].astype(str)

estoque = estoque.drop_duplicates('objeto de locação', keep='first')

estoque = estoque[['objeto de locação','IPC Previsto','Baldeio','% Baldeio', '% Transporte']]

In [1096]:
estoque['% Baldeio'] = np.where(((estoque['Baldeio'] / estoque['IPC Previsto']) * 100) > 100, 100 , ((estoque['Baldeio'] / estoque['IPC Previsto']) * 100))

In [1097]:
pas = pd.read_excel(path_pas,skiprows=1, usecols=['Id Projeto','Talhão','Data liberação ajustada']) 

pas['Data liberação ajustada'] = pd.to_datetime(pas['Data liberação ajustada'], errors='coerce')

pas = mascara(pas,'Id Projeto',4)

pas = mascara(pas,'Talhão',3)

pas['objeto de locação'] = pas['Id Projeto'] + pas['Talhão']

pas = pas.drop_duplicates('objeto de locação',keep='first')

pas = pas[['objeto de locação','Data liberação ajustada']]

In [1098]:
print(estoque.shape)
estoque.head(3)

(7838, 5)


,objeto de locação,IPC Previsto,Baldeio,% Baldeio,% Transporte
0,0265007,2457,2457,100.0,105.77
1,0903031,5758,5758,100.0,85.84
2,1067003,12052,12052,100.0,80.98


In [1099]:
#Econtrar as datas de CTO para cada fazenda

fzd_sem_cto = bd[pd.isna(bd['Data CTO Baldeio'])]

fzd_sem_cto = fzd_sem_cto['Id Projeto'].unique().tolist()

print(f'Nº de Fazendas sem cto: {len(fzd_sem_cto)}')


Nº de Fazendas sem cto: 30


In [1100]:
#trazer o retalhonamento para a base


#Encontrando o retalhonamento
bd = bd.merge(ret, left_on=['CHAVE'], right_on=['objeto de locação origem'], how = 'left').drop('objeto de locação origem', axis=1)

#Encontrando a data de CTO
bd = bd.merge(cto[['objeto de locação','data_cto']].drop_duplicates('objeto de locação',keep='first'), left_on=['objeto de locação retalhonado'], right_on=['objeto de locação'], how='left').drop('objeto de locação',axis=1)

#Atribuindo data de CTO quando estiver vazia
bd['Data CTO Baldeio'] = np.where(pd.isna(bd['Data CTO Baldeio']) , bd['data_cto'], bd['Data CTO Baldeio'])

#Encontrando o estoque do talhão
bd = bd.merge(estoque, left_on=['CHAVE'], right_on=['objeto de locação'], how='left').drop('objeto de locação',axis=1)

#Atribuindo o PAS a base
bd = bd.merge(pas,left_on=['CHAVE'],right_on=['objeto de locação'], how='left')

bd['Data CTO Baldeio'] = np.where(pd.isna(bd['Data CTO Baldeio']), bd['Data liberação ajustada'], bd['Data CTO Baldeio'])

bd['Data CTO Baldeio'] = pd.to_datetime(bd['Data CTO Baldeio'])

In [1101]:
#Dias úteis

premissa_dict = [
    {'EPS': 'Bracell', 'Dias_uteis': 25, 'Mês': 'RF12'},
    {'EPS': 'Terceiros', 'Dias_uteis': 21, 'Mês': 'RF12'},
    {'EPS': 'Bracell', 'Dias_uteis': 24, 'Mês': 'RF01'},
    {'EPS': 'Terceiros', 'Dias_uteis': 21, 'Mês': 'RF01'}
]

premissa = pd.DataFrame(premissa_dict).astype({'Dias_uteis': 'int'})


In [1102]:
#Agrupar volumes para encontrar a capacidade
bd_agg = {'Área(ha)': 'sum'}

bd_cap = bd.groupby(['Regional Excecução', 'EPS PLANTIO', 'RF PLANTIO_OFICIAL']).agg(bd_agg).reset_index()

bd_cap = bd_cap.sort_values(['RF PLANTIO_OFICIAL','Regional Excecução', 'Área(ha)'], ascending=[False, False, True])


bd_cap_rename = {'Regional Excecução': 'regional',
                 'EPS PLANTIO' : 'eps plantio',
                 'RF PLANTIO_OFICIAL': 'rf plantio oficial',
                 }

bd_cap = bd_cap.rename(bd_cap_rename, axis=1)

bd_cap['Classe equipe'] = np.where(bd_cap['eps plantio'] == 'Bracell 1', 'Bracell', 'Terceiros') 

bd_cap = bd_cap.merge(premissa, left_on=['Classe equipe', 'rf plantio oficial'], right_on=['EPS', 'Mês'], how='left').drop(['EPS', 'Mês'], axis=1)

bd_cap['Capacidade ha/dia'] = round(bd_cap['Área(ha)'] / bd_cap['Dias_uteis'],2)

bd_cap.tail()

,regional,eps plantio,rf plantio oficial,Área(ha),Classe equipe,Dias_uteis,Capacidade ha/dia
25,NOROESTE,GERAÇÃO,RF01,216.40,Terceiros,21,10.30
26,NOROESTE,CARPELO,RF01,379.64,Terceiros,21,18.08
27,CENTRO,NILZA,RF01,329.91,Terceiros,21,15.71
28,CENTRO,SOLLUM,RF01,475.11,Terceiros,21,22.62
29,BSP,Bracell 1,RF01,500.47,Bracell,24,20.85


In [1103]:
#Classe Baldeio
#0-30
#30-60
#60-90
#>90


In [ ]:
#Agrupando as fazendas para iniciar o sequenciamento

agg_bd_data = {'Data CTO Baldeio': ['min', 'max'],
               'Área(ha)': 'sum',
               }



bd_seq = bd.groupby(['RF PLANTIO_OFICIAL','Regional Excecução','Id Projeto', 'Projeto', 'EPS PLANTIO']).agg(agg_bd_data).reset_index()

bd_seq.columns = ['RF PLANTIO_OFICIAL', 'Regional Excecução', 'Id Projeto', 'Projeto', 'EPS PLANTIO', 'Baldeio Min', 'Baldeio Max', 'Área(ha)']



#Encontrando as fazendas que se repetem em outros meses

bd_seq['index_fazenda'] = bd_seq.groupby(['Projeto','EPS PLANTIO']).transform('size')


#Definindo as prioridades para sequenciamento
bd_seq = bd_seq.sort_values(['RF PLANTIO_OFICIAL','Regional Excecução','EPS PLANTIO','Baldeio Min','Projeto', 'Área(ha)'], ascending=[False,True,True,True,True,True])


#Definindo a ordem das operações
bd_seq['Ordem de Operação'] = bd_seq.groupby(['RF PLANTIO_OFICIAL','Regional Excecução', 'EPS PLANTIO']).cumcount()+1


In [1105]:
#Trazendo capacidade para a base de sequenciamento

bd_seq = bd_seq.merge(bd_cap[['eps plantio', 'regional', 'rf plantio oficial', 'Capacidade ha/dia', 'Dias_uteis']], 
                      left_on=['EPS PLANTIO','Regional Excecução', 'RF PLANTIO_OFICIAL'], 
                      right_on=['eps plantio', 'regional', 'rf plantio oficial'], 
                      how='left').drop(['eps plantio', 'regional', 'rf plantio oficial'],axis=1)

In [1106]:
#Sempre arredondar para cima o número de dias para finalizar operação
bd_seq['dias de operação'] = np.ceil(bd_seq['Área(ha)'] / bd_seq['Capacidade ha/dia']).astype(int)

bd_seq['dias de operação acumulado'] = bd_seq.groupby(['RF PLANTIO_OFICIAL','EPS PLANTIO'])['dias de operação'].cumsum()

#Definindo o deslocamento
bd_seq['Dias para deslocamento'] = 1

#Definindo o percentual que a operação ficará na fazenda
bd_seq['% de dias de operação'] =  (bd_seq['dias de operação'] / bd_seq['Dias_uteis']) #* 100).map("{:.1f}%".format)

In [1107]:
#Definindo o Início do mês operacional

de_para_im = {'RF12' : pd.Timestamp(day=21, month=11, year=2024), 
             'RF01' : pd.Timestamp(day=21, month=12, year= 2024)
}

bd_seq['Inicio de Operação'] = bd_seq['RF PLANTIO_OFICIAL'].map(de_para_im)

bd_seq['Inicio de Operação'] = pd.to_datetime(bd_seq['Inicio de Operação'])

In [1108]:
#Fim da Operação


bd_seq['Final de operação'] = bd_seq.apply(


    lambda x: np.busday_offset(
        x['Inicio de Operação'].strftime('%Y-%m-%d'), 
        x['dias de operação acumulado'],
        weekmask= '1111110',
        roll='forward'
    ) 
    
    if x['EPS PLANTIO'] == 'Bracell 1' else 
    
    np.busday_offset(
        x['Inicio de Operação'].strftime('%Y-%m-%d'), 
        x['dias de operação acumulado'],
        weekmask= '1111100',
        roll='forward')
    
   
    
    
    , axis=1 
)

In [1109]:
bd_seq ['Inicio de Operação'] = np.where( bd_seq['Ordem de Operação'] == 1 , bd_seq['Inicio de Operação'],
                                          
                                          ( bd_seq['Final de operação'].shift(1))
                                          )


In [1110]:
#Entrando quantos hectares estão estão sem data de CTO

bd_seq.to_excel('l - bd1.xlsx',index=False)

In [1084]:
#Já sabendo capacidade, agora dividir e redistruir a ordem das fazendas de acordo com a data mínima de baldeio por eps
#Agrupar por fazenda considerando a data mínima de liberação


    #1º Classificar qual é a primeira fazenda do mês para cada EPS



In [1085]:
#Classe Baldeio
#0-30
#30-60
#60-90
#>90




In [ ]:
#Antes de apelar para definir dias, verificar se o PAS não tem data de liberação

    #O pas tem todas as datas de liberação, não é necessário utilizar a base para definir o dia com base no volume baldeado


#Definindo data se o baldeio caso ele já tenha sido iniciado


# percent_b_90 = pd.Timestamp(day=12, month=11, year=2024)

# #Poderia utilizar a capacidade, porém não necessáriamente estariam baldeando na determinada fazenda

# percent_b_b90 = percent_b_90 + pd.Timedelta(days=15)


# bd['Data CTO Baldeio'] = np.where(
#     (pd.isna(bd['Data CTO Baldeio'])) & (bd['% Baldeio'] >= 80), percent_b_90,
#     np.where(
#         (pd.isna(bd['Data CTO Baldeio'])) & (bd['% Baldeio'] < 80), percent_b_b90,
#         bd['Data CTO Baldeio']
#     )
# )


# bd['Data CTO Baldeio'] = pd.to_datetime(bd['Data CTO Baldeio'])